In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
import re
import json
import spacy
import os
import requests
import sys
import deepl
import collections
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import svm

In [119]:
nlp = spacy.load('en_core_web_sm')

Structuring the text:

The following functions provide a way to get raw text and categorize it into the different figure numbers, the different sentences the subject spoke for each figure as well as storing information like the special position markers associated with the specific sentences they refer to.

In [120]:
raw_text = '''

I/ @ ^ Une sorte de coléoptère avec des grandes ailes
ou ça peut être une sorte de grenouille.
V Une sorte de masque Viking avec quatre yeux.
Une sorte de silhouette.
Un petit personnage avec  une bouche ouverte qui a l’air surpris
et son reflet là.


II/ V Deux personnages en train d’établir un contact ou
un combat parce qu’il y a du sang ou une jambe qui a été coupée.
Des poumons.
^ Une tête de coq avec la crête
Avion de chasse, planeur, feu des réacteurs ;
là aussi ça peut être un plus grand qui envoie des missiles
Des hommes troncs, peut-être irrationnels
ou des espèces de chenilles avec un gros ventre.

III/ ^ Deux guitares avec des formes bizarres, le manche
tordu.
Encore deux personnages face à face bizarrement constitués,
comme du Picasso. Ou deux personnages qui font qu’un.
Deux espèces de missiles ou de torpilles.
Des poumons
ou un papillon.
V Juste la partie d’un vieillard avec sa canne
ou un embryon.
Une espèce de grosse bête, on voit l’intérieur de son corps,
Les mandibules.

IV/ V Je vois encore une sorte de créature bizarre, un colosse,
Avec  les ailes là et des pattes étranges.
^ Deux personnages sous une espèce de burka.
V La tête de la mouche (film).
> Tête de bébé crocodile ou de cochon vu qu’il y a pas d’oreille
ou peut-être une taupe, une sorte de trompe.

V/ ^ Je vois aussi une sorte de mélange entre une chauve-souris
et un escargot, là les pattes sont un peu étranges.
@ Sinon là je vois rien d’autre.
Peut-être deux têtes de personnages en tout petit,
Ils sont dos à dos.
Le guitariste d’ACDC : sa tête avec ses cheveux.

VI/ @ ^ Une libellule un peu bizarre avec des ailes
à demi détruites.
Là, la mâchoire d’une bête avec des crochets à l’intérieur
de sa bouche, mais y a pas d’œil alors c’est peut-être
une plante carnivore.
L’homme sable (dans Spiderman) en train de reprendre
forme humaine.
Sinon c’est tout ce que je vois.
Ou encore des poumons qui sont cachés juste là
ou des sortes de graines de flageolets.
Ou là une sorte de personnage avec quelque chose
pour faire des bonds.
Finalement, c’est peut-être une sorte d’ange
extraterrestre qui va dans une île parce qu’il y a des ailes.
Il descend par là donc finalement c’est peut-être
un enfant qui va venir au monde.

VII/ V Deux espèces d’hippocampes avec une trompe ici ;
ils sont reliés les deux par quelque chose qui est là ;
ils ont l’air d’être congelés parce qu’il y a des stalactites,
comme peut-être des animaux préhistoriques qui ont été congelés.
^ Ou là sinon on peut voir des espèces de personnages
avec des bras en forme de tronçonneuse qui tentent de se rapprocher.
Des soldats de l’armée prussienne avec des pointes sur le casque.
Là on peut voir une espèce de tête avec un groin mais vu
qu’il y a une proéminence ça peut être le bouffon du roi.
Si on intègre le blanc on peut voir un masque.
V Une arme des Mayas ; ou peut-être pas une arme,
un objet Maya pour le culte
ou une espèce de champignon.

VIII/ V Une vue de microscope avec les colorants qu’il faut.
Deux espèces d’insectes qui sont en train de butiner ce qui est là.
Une tête de mouche avec quelque chose qui dégouline (orange).
Des cordes vocales (traits verts).
Un personnage avec des sortes d’antennes, en faisant
abstraction des cordes vocales.
Deux caméléons qui gravissent une montagne ;
on a l’impression qu’ils se serrent une main ;
il y a une transaction ou quelque chose comme ça :
ils donnent de l’argent ou se rechargent en énergie.

IX/ ^ Deux espèces de créatures difformes qui se rapprochent.
Deux narines, peut-être un crane qui est caché par tout le reste.
Une tête de fantôme en partie cachée par le reste
ou une sorte de champ magnétique.
Deux îles ou deux pays qui sont le reflet de l’autre (vert).
Un morceau de jambon de Parme (rose).
Des espèces de racines, comme le logo de Sépultura (traits oranges)
Une espèce de matière symbiotique qui fusionne.
Un personnage avec des ailes (vert).

X/ ^ Ca fait assez art contemporain, comme du Miro.
Deux espèces de petites bestioles qui se battent pour savoir
ce qui est au milieu (noir).
Des pieuvres ou gendarmes extraterrestres qui essayent
de les séparer (bleu).
Deux petits soleils ou comètes (jaune).
Deux chenilles ou la lettre Omega avec une araignée ou
diapason au boût (vert).
Deux sortes d’embryons qui recrachent une matière bleue (rose).
Deux espèces de fourmis avec une flamme (noir+jaune), elles sont
entrain de rattraper les deux choses qui sont là (orange)
ou la moto du Ghostrider.
V Je vois aussi deux yeux avec une bouche bizarre :
le vert c’est les cornes, le bleu la bouche, le gris un rasoir
électrique ou une sorte de tournevis sonique.
Deux personnages avec un ponpon bleu qui sont
entrain de se liquéfier, ils sont peut-être constitués de peau
et d’air (rose).
Les deux fourmis, elles vont essayer de mettre un chapeau
pour les reboucher pour pas qu’elles se dégonflent.
Deux brosses pour le dos ou à cheveux ou un compas
un peu bizarre (orange).
Finalement les deux chenilles il y a leurs projections astrales
qui est entrain de sortir de leur corps.
Deux personnes qui se soutiennent pour pas perdre l’équilibre
et pas tomber là (bleu central).

Choix + : X parce qu’on peut voir beaucoup de choses.
Choix - : II parce que ça a l’air de deux personnes qui sont entrain de se battre à mort, deux gladiateurs. Puis il y a aussi peut-être les espèces d’avions de guerre.


Rq : latences longues sauf à V et VIII
'''

# 1. Organizing the text into the corresponding figures and responses

Since the original text is in French but most of the resources are English, doing the translation after extracting the position markers may be more appropiate since the automatic translation may mess with those caracters, specially the V and mistranslate them. We are using the DeepL free API which allows for 500.000 characters a month.

Another note on DeepL: there are a lot of "figures" mentioned in the target text, that is, the translation. If you look at the prompt section, the llm seems to confuse it by figures as in art figures instead of figures as in human figures. I know that when doing the DeepL API call you can pass a glossary as an argument to the function. Maybe I would be a good idea to make a glossary with more concrete terms if there are synonyms to those semantically ambiguous nouns. Alternative: look for a way to measure the specificity of a word to deal with those kinds of nouns/terms

This part is commented out because there is no need to abuse the API limit, the ouput dictionary was saved as demo_text_preprocess so we can keep working on the different parts of the project.

As you can see, we are doing most of the parsing with regular expressions. The flow of the function is:
- 1st we split the text according to the roman numerals and we extract the Choix and Rq parts.
- 2nd we clean up the response text to avoid unnecesay extralines and we extract the special markers. At this point we have two full text variables: one raw that we will extract the positional markers from and a clean copy we'll send to DeepL.
- 3rd the clean text for each figure goes through the DeepL API and we store the translation for the whole text corresponding to the figure.
- 4th we split the text corresponding to each figure into its individual sentences.

In [121]:
def get_responses(raw_text):
  
  """This function takes plain text, parses the in-text markers to extract the text corresponding to each Rorschach figure,
  runs the responses through the Deepl API to translate them and returns a dictionary with the responses for each figure properly organized.
  """

  dict_numbers = {"I": 1, "II": 2,
                  "III": 3, "IV": 4,
                  "V": 5, "VI": 6,
                  "VII": 7, "VIII": 8,
                  "IX": 9, "X": 10}

  regex_response_number = '[A-Z]+/'                                # since the responses alway start with the number + \

  list_number_figure = re.findall(regex_response_number, raw_text)  # finds the regex for the number of responses

  text_i = re.split(regex_response_number, raw_text)

  # regex_choix_pos = r'Choix +(.*?)Choix -'

  # regex_choix_neg = r'Choix -(.*?)Rq :'

  # regex_rq = r'Rq :(.*?)\n'

  try: 
    
    text_choix_pos = re.findall(r'Choix \+ :(.*?)\nCh.?', raw_text)[0].strip()

    text_choix_neg = re.findall(r'Choix \- :(.*?)\n', raw_text)[0].strip()

    text_rq = re.findall(r'Rq :(.*?)\n', raw_text)[0].strip()

    additional_info = {"Choix_pos": text_choix_pos, "Choix_neg": text_choix_neg, "Rq": text_rq}
    
  except:
      
      additional_info = {"Choix_pos": "nada", "Choix_neg": "nada", "Rq": "nada"}
      
      print("Additional info not found")


  list_responses = []

  for i in range(len(list_number_figure)):

    if list_number_figure[i] not in text_i[i]:

      j = i + 1 # there is a \n\n string at index 0 so the text actually starts at index 1
      # print("not found")

    else:

      j = i

    dict_responses = {}

    n_response = list_number_figure[i][:-1]

    number = dict_numbers[n_response]

    text_i = re.split(regex_response_number, raw_text)

    full_text = text_i[j].strip()

    regex_line_break = '\n++'

    list_sentences_raw = text_i[j].split(".")

    text_i[j] = re.sub(regex_line_break, " ", text_i[j]) \
                  .replace("@", "") \
                  .replace("^", "") \
                  .replace("V", "") \
                  .replace(">", "") \
                  .replace("  ", " ") \
                  .strip() \

    # DEEPL API TRANSLATION

    with open("DEEPL API KEY.txt", "r") as f:
      API_KEY = f.read()

    translator = deepl.Translator(API_KEY)
    result = translator.translate_text(text_i[j], target_lang="EN-US", preserve_formatting=True)

    text_i[j] = result.text

    dict_responses["figure_number"] = number

    dict_responses["raw_response"] = full_text

    dict_responses["clean_response"] = text_i[j]

    list_sentences_clean = text_i[j].split(".")

    clean_sentences = []

    i = 0

    special_markers_list = ["@", "^", "V ", "<", ">",]

    # structuring the sentences inside the while loop

    while i < len(list_sentences_clean):

      if "Choix" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break

      elif "Choice" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break

      j = i + 1

      # cleaning the sentences and adding the . back at the end of the sentence

      dict_sentence_info = {}

      # use the full_text of the text here to parse, the text[j is clean
      item = list_sentences_clean[i].strip()

      item_2 = list_sentences_raw[i].strip()

      if item != "":                      # split leaves an empty string at the end of the list and by adding a . , we get a "." item at the end of the list
        item = item + "."
        dict_sentence_info["response_{}".format(j)] = item
        clean_sentences.append(dict_sentence_info)


        for marker in special_markers_list:
          if marker in item_2:
            dict_sentence_info["special_marker"] = marker

      j = j + 1

      i = i + 1

    dict_responses["sentences"] = clean_sentences
        # in case we need to double-check

    list_responses.append(dict_responses)

  list_responses.append(additional_info)



  return list_responses

In [122]:
responses = get_responses(raw_text)
responses

[{'figure_number': 1,
  'raw_response': '@ ^ Une sorte de coléoptère avec des grandes ailes\nou ça peut être une sorte de grenouille.\nV Une sorte de masque Viking avec quatre yeux.\nUne sorte de silhouette.\nUn petit personnage avec  une bouche ouverte qui a l’air surpris\net son reflet là.',
  'clean_response': 'Some kind of beetle with big wings, or it could be some kind of frog. Some kind of iking mask with four eyes. Some kind of silhouette. A small character with an open mouth who looks surprised and his reflection there.',
  'sentences': [{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'special_marker': '^'},
   {'response_2': 'Some kind of iking mask with four eyes.',
    'special_marker': 'V '},
   {'response_3': 'Some kind of silhouette.'},
   {'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'}]},
 {'figure_number': 2,
  'raw_response': 'V Deux personnages en train d’établir un contac

In [123]:
with open("demo_text_preprocess", "w") as f:
    json.dump(responses, f)

The proper clean single sentence responses in English are in the dictionary here: [{sentences: [{response_x}]}]

In [124]:
demo = responses
demo

[{'figure_number': 1,
  'raw_response': '@ ^ Une sorte de coléoptère avec des grandes ailes\nou ça peut être une sorte de grenouille.\nV Une sorte de masque Viking avec quatre yeux.\nUne sorte de silhouette.\nUn petit personnage avec  une bouche ouverte qui a l’air surpris\net son reflet là.',
  'clean_response': 'Some kind of beetle with big wings, or it could be some kind of frog. Some kind of iking mask with four eyes. Some kind of silhouette. A small character with an open mouth who looks surprised and his reflection there.',
  'sentences': [{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'special_marker': '^'},
   {'response_2': 'Some kind of iking mask with four eyes.',
    'special_marker': 'V '},
   {'response_3': 'Some kind of silhouette.'},
   {'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'}]},
 {'figure_number': 2,
  'raw_response': 'V Deux personnages en train d’établir un contac

# 2 Filtering the responses from the dictionary

This next couple of functions just take the previous dictionary and extracts just the sentences. It "shaves off" all the unnecesary fluff which will not be useful to process the text.

In [125]:
def get_list_figure_responses(og_dict):

    # if you look at the depth of the sentences in the original dictionary,
    # the responses are a list of dictionaries of structure [{response: blablabla, special_marker: blablabla}, {response_2: blablabla, special_marker: blablabla}]
    # to be able to extract the sentences to process them and do the computations that we need to do,
    # i thought it would be nicer to have them without all the fluff of the other dictionary keys, etc.
    # I am just going to make this function to get the 10 figures into 10 dataframes with its responses and drop all the other stuff we dont need
    
    

    responses_list = []

    for figure in og_dict[:-1]:

        figure_sents = figure["sentences"]

        for item in figure_sents:

            item.pop("special_marker", None)
        responses_list.append(figure_sents)

    return responses_list

def clean_dict(responses_fig_list):

    '''merges the dictionary for each the responses of a figure into one response
    to go from a list of dicts [{response: blablabla},  {response: blablabla}, {response: blablabla}]
    to just one dict{response_1: , response_2: , response_3: }

    '''

    new_dict = {}
    for dictionary in responses_fig_list:
            for k , v in dictionary.items():
                # print(k,v)
                new_dict[k]= v
    return new_dict

# def figure_df(clean_dict):
#     '''gets the previous clean dict and turns it into a dataframe
#     '''
#     clean_responses_df = pd.DataFrame.from_dict(clean_dict, orient='index')
#     return clean_responses_df


In [126]:
def transform_dictionary_to_figure_dataframes(og_dictionary):
    '''this function transforms the first dictionary
    into individual response dictionaries for each figure and returns it in list form.
    '''

    responses_list = get_list_figure_responses(og_dictionary)

    clean_fig_1 = clean_dict(responses_list[0])
    clean_fig_2 = clean_dict(responses_list[1])
    clean_fig_3 = clean_dict(responses_list[2])
    clean_fig_4 = clean_dict(responses_list[3])
    clean_fig_5 = clean_dict(responses_list[4])
    clean_fig_6 = clean_dict(responses_list[5])
    clean_fig_7 = clean_dict(responses_list[6])
    clean_fig_8 = clean_dict(responses_list[7])
    clean_fig_9 = clean_dict(responses_list[8])
    clean_fig_10 = clean_dict(responses_list[9])

    list_dicts = [clean_fig_1, clean_fig_2, clean_fig_3, clean_fig_4, clean_fig_5, clean_fig_6, clean_fig_7, clean_fig_8, clean_fig_9, clean_fig_10]

    return list_dicts

The output of the functions look something like this:

In [127]:
list_dicts = transform_dictionary_to_figure_dataframes(demo)
list_dicts

[{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
  'response_2': 'Some kind of iking mask with four eyes.',
  'response_3': 'Some kind of silhouette.',
  'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'},
 {'response_1': "Two characters making contact or fighting because there's blood or a leg has been cut off.",
  'response_2': 'Lungs.',
  'response_3': "A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly."},
 {'response_1': 'Two oddly shaped guitars with twisted necks.',
  'response_2': 'Again, two figures face to face, strangely constituted, like Picasso.',
  'response_3': 'Or two figures as one.',
  'response_4': 'Two kinds of missiles or torpedoes.',
  'response_5': 'Lungs or a butterfly.',
  'response_6': 'Just part of an old man with his cane, or an e

In [128]:
with open("list_responses_clean.json", "w") as f:
    json.dump(list_dicts, f)

# Models

Classification function 1: Contents

In [129]:
data=pd.read_csv('rap_dataset.tsv', sep='\t')

In [130]:
df1=data.dropna()
df1

,User,PQLevel,Client,Age,Gender,Country,Protocol,TestDate,R,ResponseOrder,...,Determinants,Pair,FormQuality,FQText,Contents,Popular,ZCode,ZScore,SpecialScores,Rejection
121,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,1,...,Ma,1,u,Animal (Winged but unspecified),A,0,ZW,1.0,"FAB,PHR",0
129,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,9,...,Mp,0,u,Face (Photo negative and includes use of DS7 o...,(H),0,ZW,2.5,GHR,0
130,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,10,...,"Ma,FC",0,o,Flower,Bt,0,ZW,4.5,GHR,0
144,272,C,41287,10,Male,Israel,42274,6/16/2018 14:52,18,9,...,F,0,o,Human-Like Figure,(H),1,ZW,2.0,GHR,0
155,272,C,42937,15,Male,Afghanistan,43935,9/10/2018 6:36,20,2,...,"Mp,Fr",0,o,Humans (Two),H,0,ZA,4.0,GHR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506392,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,8,...,"Fr,FMp",0,o,Monkey,A,0,ZA,3.0,"COP,GHR",0
506395,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,11,...,FT,0,o,Moth,A,0,ZW,1.0,PER,0
506398,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,14,...,"FV,FMa",1,-,"Shrimp (Two, natural form)","A,Na",0,ZW,2.5,"COP,PHR",0
506451,29942,B,58333,24,Not Specified,Georgia,59491,9/12/2020 15:32,8,7,...,"FD,CF",0,-,Face,(Hd),0,ZW,5.5,"PHR,ALOG",0


In [131]:
X = df1['FQText'] #we create a NumPy array with the text/answer
y = df1['Contents'] #other array for the coding of this answer

In [132]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [133]:
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42) #Change test_size


In [134]:
y_train

47932             A
398065        Hd,Ay
453400            H
30894           (H)
119074    (H),Cg,Sc
            ...    
177786           Hd
477196            A
182094     (Hd),Art
14857        (H),Cg
232260          (H)
Name: Contents, Length: 12198, dtype: object

In [135]:
X_train

47932          Animals (Two, meeting criterion for o as D1)
398065                                         Face (Clown)
453400                         Humans or Human-Like Figures
30894                  Face (Monster, gremlin, evil, alien)
119074                                                Witch
                                ...                        
177786                         Humans or Human-Like Figures
477196                                                  Bat
182094    Mask (Note: This category includes a wide vari...
14857                                     Human-Like Figure
232260                          Humans (Two facing midline)
Name: FQText, Length: 12198, dtype: object

In [136]:
X_train.shape

(12198,)

In [137]:
y_train.shape

(12198,)

In [138]:
from sklearn.metrics import f1_score

# text_preprocess = ColumnTransformer(transformers=[("tfidf", TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3)), "FQText")], 
#                                     remainder="passthrough")

pipeline = Pipeline([
                ('text_tfidf', TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3))),
                ('clf', svm.LinearSVC(dual=True, max_iter=1000)),
            ])

pipeline.fit(X_train, y_train)

accuracy = pipeline.score(X_test,y_test)

y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred, average="micro")

print(accuracy, f1)

0.48583372039015327 0.48583372039015327


In [139]:
# exporting the model and tdidf

import pickle

pickle.dump(pipeline, open("svm_contents_V1-03-04.sav", 'wb'))



FX to classify content

In [140]:
def classify_contents(text, model_contents="svm_contents_V1-03-04.sav"):
    
    clf = pickle.load(open(model_contents, "rb"))
    
    prediction = clf.predict([text])[0]
    
    return prediction

Classify function 2: Determinants

In [141]:
X = df1['FQText'] #we create a NumPy array with the text/answer
y = df1['Determinants'] #other array for the coding of this answer

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42) #Change test_size

# text_preprocess = ColumnTransformer(transformers=[("tfidf", TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3)), "FQText")], 
#                                     remainder="passthrough")

pipeline = Pipeline([
                ('text_tfidf', TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3))),
                ('clf', svm.LinearSVC(dual=True, max_iter=1000)),
            ])

pipeline.fit(X_train, y_train)

accuracy = pipeline.score(X_test,y_test)

y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred, average="micro")

print(accuracy, f1)


0.3808639108221087 0.3808639108221087


In [142]:
import pickle

pickle.dump(pipeline, open("svm_determinants_V1-03-04.sav", 'wb'))

In [143]:
def classify_determinants(text, model_determinants="svm_determinants_V1-03-04.sav"):
    
    clf_determinants = pickle.load(open(model_determinants, "rb"))
    
    prediction = clf_determinants.predict([text])[0]
    
    return prediction

In [144]:
list_dicts[0]

{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
 'response_2': 'Some kind of iking mask with four eyes.',
 'response_3': 'Some kind of silhouette.',
 'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'}

In [145]:
classify_contents(list_dicts[0]["response_1"])

'A'

In [146]:
classify_determinants(list_dicts[0]["response_1"])

'F'

# Adapting the classify model to the way we are storing our info

In [147]:
def eval(list_dicts):
    """This function runs the evaluation with our first two models. 
    It takes as input the list of dictionary responses, prints the evaluation 
    and returns the content and determinant labels for each response in dictionary form.   
    """
    evaluation = []
    i = 1
    for dictionary in list_dicts:
        
        figure_number = "Figure_{}".format(i)
        
        dict_evaluation_per_figure = {}
        
        print(figure_number, "\n") 
        
        list_evaluation_figure = []
        
        j = 1
        
        for response in dictionary:
            
            sentence = dictionary[response]
            # print(dictionary[response]) #works until here
            
            content = classify_contents(dictionary[response])
            
            # print(response, content)
            
            determinant = classify_determinants(dictionary[response])
            
            # print(response, content, determinant)
            
            dict_eval = {}
            dict_eval["response"] = sentence
            dict_eval["content"] = content
            dict_eval["determinant"] = determinant
            list_evaluation_figure.append(dict_eval)
            
            print("Response {}: ".format(j), sentence, "\nContent:", content , "\nDeterminant:", determinant, "\n")
            
            j = 1 + j
            
        dict_evaluation_per_figure[figure_number] = list_evaluation_figure
        
        evaluation.append(dict_evaluation_per_figure)
        
        i = i +1
        
    return evaluation

In [148]:
evaluation = eval(list_dicts)

Figure_1 

Response 1:  Some kind of beetle with big wings, or it could be some kind of frog. 
Content: A 
Determinant: F 

Response 2:  Some kind of iking mask with four eyes. 
Content: Hd 
Determinant: F 

Response 3:  Some kind of silhouette. 
Content: A 
Determinant: F 

Response 4:  A small character with an open mouth who looks surprised and his reflection there. 
Content: Hd 
Determinant: F 

Figure_2 

Response 1:  Two characters making contact or fighting because there's blood or a leg has been cut off. 
Content: Hd 
Determinant: F 

Response 2:  Lungs. 
Content: An 
Determinant: FC 

Response 3:  A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly. 
Content: (H),Bt 
Determinant: Mp 

Figure_3 

Response 1:  Two oddly shaped guitars with twisted necks. 
Content: H 
Determinant: Ma 

Response 2:  Again, two figures face to face, strangely

Response 6:  Some kind of roots, like the Sepultura logo (orange lines) Some kind of symbiotic matter that fuses together. 
Content: A 
Determinant: F 

Response 7:  A character with wings (green). 
Content: (H) 
Determinant: F 

Figure_10 

Response 1:  It's quite contemporary art, like Miro. 
Content: Art 
Determinant: F 

Response 2:  Two species of little bugs fighting over what's in the middle (black). 
Content: H,Cg 
Determinant: F 

Response 3:  Octopuses or alien policemen trying to separate them (blue). 
Content: Sc,Fi,Na 
Determinant: Ma 

Response 4:  Two little suns or comets (yellow). 
Content: Hd,Hx 
Determinant: rF,Mp 

Response 5:  Two caterpillars or the letter Omega with a spider or tuning fork at the tip (green). 
Content: A 
Determinant: F 

Response 6:  Two kinds of embryos spitting out blue matter (pink). 
Content: (H) 
Determinant: Mp 

Response 7:  Two kinds of ants with a flame (black+yellow), catching up with the two things there (orange) or the Ghostrider's m

In [149]:
evaluation

[{'Figure_1': [{'response': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'content': 'A',
    'determinant': 'F'},
   {'response': 'Some kind of iking mask with four eyes.',
    'content': 'Hd',
    'determinant': 'F'},
   {'response': 'Some kind of silhouette.',
    'content': 'A',
    'determinant': 'F'},
   {'response': 'A small character with an open mouth who looks surprised and his reflection there.',
    'content': 'Hd',
    'determinant': 'F'}]},
 {'Figure_2': [{'response': "Two characters making contact or fighting because there's blood or a leg has been cut off.",
    'content': 'Hd',
    'determinant': 'F'},
   {'response': 'Lungs.', 'content': 'An', 'determinant': 'FC'},
   {'response': "A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly.",
    'content': '(H),Bt',
    'determinant': 'Mp'}]},
 {'Figure_

In [150]:
classify_determinants("dog")

'FMa'

# 4 PUTTING IT ALL TOGETHER

In [151]:
raw_text = '''

I/ @ ^ Une sorte de coléoptère avec des grandes ailes
ou ça peut être une sorte de grenouille.
V Une sorte de masque Viking avec quatre yeux.
Une sorte de silhouette.
Un petit personnage avec  une bouche ouverte qui a l’air surpris
et son reflet là.


II/ V Deux personnages en train d’établir un contact ou
un combat parce qu’il y a du sang ou une jambe qui a été coupée.
Des poumons.
^ Une tête de coq avec la crête
Avion de chasse, planeur, feu des réacteurs ;
là aussi ça peut être un plus grand qui envoie des missiles
Des hommes troncs, peut-être irrationnels
ou des espèces de chenilles avec un gros ventre.

III/ ^ Deux guitares avec des formes bizarres, le manche
tordu.
Encore deux personnages face à face bizarrement constitués,
comme du Picasso. Ou deux personnages qui font qu’un.
Deux espèces de missiles ou de torpilles.
Des poumons
ou un papillon.
V Juste la partie d’un vieillard avec sa canne
ou un embryon.
Une espèce de grosse bête, on voit l’intérieur de son corps,
Les mandibules.

IV/ V Je vois encore une sorte de créature bizarre, un colosse,
Avec  les ailes là et des pattes étranges.
^ Deux personnages sous une espèce de burka.
V La tête de la mouche (film).
> Tête de bébé crocodile ou de cochon vu qu’il y a pas d’oreille
ou peut-être une taupe, une sorte de trompe.

V/ ^ Je vois aussi une sorte de mélange entre une chauve-souris
et un escargot, là les pattes sont un peu étranges.
@ Sinon là je vois rien d’autre.
Peut-être deux têtes de personnages en tout petit,
Ils sont dos à dos.
Le guitariste d’ACDC : sa tête avec ses cheveux.

VI/ @ ^ Une libellule un peu bizarre avec des ailes
à demi détruites.
Là, la mâchoire d’une bête avec des crochets à l’intérieur
de sa bouche, mais y a pas d’œil alors c’est peut-être
une plante carnivore.
L’homme sable (dans Spiderman) en train de reprendre
forme humaine.
Sinon c’est tout ce que je vois.
Ou encore des poumons qui sont cachés juste là
ou des sortes de graines de flageolets.
Ou là une sorte de personnage avec quelque chose
pour faire des bonds.
Finalement, c’est peut-être une sorte d’ange
extraterrestre qui va dans une île parce qu’il y a des ailes.
Il descend par là donc finalement c’est peut-être
un enfant qui va venir au monde.

VII/ V Deux espèces d’hippocampes avec une trompe ici ;
ils sont reliés les deux par quelque chose qui est là ;
ils ont l’air d’être congelés parce qu’il y a des stalactites,
comme peut-être des animaux préhistoriques qui ont été congelés.
^ Ou là sinon on peut voir des espèces de personnages
avec des bras en forme de tronçonneuse qui tentent de se rapprocher.
Des soldats de l’armée prussienne avec des pointes sur le casque.
Là on peut voir une espèce de tête avec un groin mais vu
qu’il y a une proéminence ça peut être le bouffon du roi.
Si on intègre le blanc on peut voir un masque.
V Une arme des Mayas ; ou peut-être pas une arme,
un objet Maya pour le culte
ou une espèce de champignon.

VIII/ V Une vue de microscope avec les colorants qu’il faut.
Deux espèces d’insectes qui sont en train de butiner ce qui est là.
Une tête de mouche avec quelque chose qui dégouline (orange).
Des cordes vocales (traits verts).
Un personnage avec des sortes d’antennes, en faisant
abstraction des cordes vocales.
Deux caméléons qui gravissent une montagne ;
on a l’impression qu’ils se serrent une main ;
il y a une transaction ou quelque chose comme ça :
ils donnent de l’argent ou se rechargent en énergie.

IX/ ^ Deux espèces de créatures difformes qui se rapprochent.
Deux narines, peut-être un crane qui est caché par tout le reste.
Une tête de fantôme en partie cachée par le reste
ou une sorte de champ magnétique.
Deux îles ou deux pays qui sont le reflet de l’autre (vert).
Un morceau de jambon de Parme (rose).
Des espèces de racines, comme le logo de Sépultura (traits oranges)
Une espèce de matière symbiotique qui fusionne.
Un personnage avec des ailes (vert).

X/ ^ Ca fait assez art contemporain, comme du Miro.
Deux espèces de petites bestioles qui se battent pour savoir
ce qui est au milieu (noir).
Des pieuvres ou gendarmes extraterrestres qui essayent
de les séparer (bleu).
Deux petits soleils ou comètes (jaune).
Deux chenilles ou la lettre Omega avec une araignée ou
diapason au boût (vert).
Deux sortes d’embryons qui recrachent une matière bleue (rose).
Deux espèces de fourmis avec une flamme (noir+jaune), elles sont
entrain de rattraper les deux choses qui sont là (orange)
ou la moto du Ghostrider.
V Je vois aussi deux yeux avec une bouche bizarre :
le vert c’est les cornes, le bleu la bouche, le gris un rasoir
électrique ou une sorte de tournevis sonique.
Deux personnages avec un ponpon bleu qui sont
entrain de se liquéfier, ils sont peut-être constitués de peau
et d’air (rose).
Les deux fourmis, elles vont essayer de mettre un chapeau
pour les reboucher pour pas qu’elles se dégonflent.
Deux brosses pour le dos ou à cheveux ou un compas
un peu bizarre (orange).
Finalement les deux chenilles il y a leurs projections astrales
qui est entrain de sortir de leur corps.
Deux personnes qui se soutiennent pour pas perdre l’équilibre
et pas tomber là (bleu central).

Choix + : X parce qu’on peut voir beaucoup de choses.
Choix - : II parce que ça a l’air de deux personnes qui sont entrain de se battre à mort, deux gladiateurs. Puis il y a aussi peut-être les espèces d’avions de guerre.


Rq : latences longues sauf à V et VIII
'''

In [152]:
def raw_text_response_eval(raw_text):
    
    responses = get_responses(raw_text)

    list_dicts = transform_dictionary_to_figure_dataframes(responses)

    evaluation = eval(list_dicts)
    
    return evaluation

In [153]:
def translated_dict_response_eval(dictionary): 
    """Use this one if you are working with the demo dictionaries already done to save deepl credits"""
    
    responses = dictionary

    list_dicts = transform_dictionary_to_figure_dataframes(responses)

    evaluation = eval(list_dicts)
    
    return evaluation

In [154]:
# raw_text_response_eval(raw_text)

In [155]:
evaluation = translated_dict_response_eval(demo) 

Figure_1 

Response 1:  Some kind of beetle with big wings, or it could be some kind of frog. 
Content: A 
Determinant: F 

Response 2:  Some kind of iking mask with four eyes. 
Content: Hd 
Determinant: F 

Response 3:  Some kind of silhouette. 
Content: A 
Determinant: F 

Response 4:  A small character with an open mouth who looks surprised and his reflection there. 
Content: Hd 
Determinant: F 

Figure_2 

Response 1:  Two characters making contact or fighting because there's blood or a leg has been cut off. 
Content: Hd 
Determinant: F 

Response 2:  Lungs. 
Content: An 
Determinant: FC 

Response 3:  A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly. 
Content: (H),Bt 
Determinant: Mp 

Figure_3 

Response 1:  Two oddly shaped guitars with twisted necks. 
Content: H 
Determinant: Ma 

Response 2:  Again, two figures face to face, strangely

Response 3:  Maybe two very small heads, back to back. 
Content: H 
Determinant: Mp 

Response 4:  The ACDC guitarist: his head with his hair. 
Content: Hd 
Determinant: F 

Figure_6 

Response 1:  A rather bizarre dragonfly with half-destroyed wings. 
Content: (H) 
Determinant: F 

Response 2:  Here, the jaw of a beast with fangs inside its mouth, but there's no eye, so maybe it's a carnivorous plant. 
Content: Hd 
Determinant: F 

Response 3:  The Sandman (in Spiderman) returning to human form. 
Content: H 
Determinant: F 

Response 4:  Otherwise, that's all I can see. 
Content: H 
Determinant: F 

Response 5:  Or lungs hidden right there, or some kind of flageolet seeds. 
Content: An 
Determinant: rF,Mp 

Response 6:  Or some kind of figure with something to leap on. 
Content: (H) 
Determinant: FT,mp,Fr,FD 

Response 7:  Finally, maybe it's some kind of extraterrestrial angel going to an island because it's got wings. 
Content: (H) 
Determinant: F 

Response 8:  He comes down that w

In [156]:
evaluation

[{'Figure_1': [{'response': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'content': 'A',
    'determinant': 'F'},
   {'response': 'Some kind of iking mask with four eyes.',
    'content': 'Hd',
    'determinant': 'F'},
   {'response': 'Some kind of silhouette.',
    'content': 'A',
    'determinant': 'F'},
   {'response': 'A small character with an open mouth who looks surprised and his reflection there.',
    'content': 'Hd',
    'determinant': 'F'}]},
 {'Figure_2': [{'response': "Two characters making contact or fighting because there's blood or a leg has been cut off.",
    'content': 'Hd',
    'determinant': 'F'},
   {'response': 'Lungs.', 'content': 'An', 'determinant': 'FC'},
   {'response': "A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly.",
    'content': '(H),Bt',
    'determinant': 'Mp'}]},
 {'Figure_